In [1]:
from PIL import Image, ImageOps
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# Load the image
image_path = "./sifnos-greece-3840x2160-12799.jpg"
image = Image.open(image_path)
image_array = np.array(image)

In [3]:
optimizer_name = 'Adam'
# optimizer_name = 'SGD'
# optimizer_name = 'LBFGS'

In [4]:
def fit_model(model, train_dataloader):
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    # optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    # optimizer = optim.LBFGS(model.parameters(), lr=0.01, max_eval=None, tolerance_grad=1e-5, line_search_fn="strong_wolfe")
    # for optimizer in optimizers:
    loss_optimizer = []
    def closure():
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)        
        loss.backward()
        return loss

    # Train the model
    num_epochs = 5
    for epoch in range(num_epochs):
        l = []
        for inputs, targets in train_dataloader:
            optimizer.step(closure)
            loss = closure()
            l.append(loss.item())
            print(f'Epoch {epoch} --> Loss : {loss}', end='\r')
        loss_optimizer.append(np.mean(l))
        print()
    
    plt.plot(loss_optimizer,label="Loss of optimizer")
    plt.xlabel('epoch')
    plt.ylabel('Cost/ total loss')
    plt.legend()
    plt.show()

    return model


def predict_model(model, coordinates, height, width, question_number, section_number, model_number):        
    # Predict RGB values for all coordinates
    batch_size = 64
    with torch.no_grad():
        predicted_rgb_values = []
        for i in range(0, coordinates.size(0), batch_size):
            batch_coords = coordinates[i:i + batch_size, :]
            batch_preds = model(batch_coords)
            predicted_rgb_values.append(batch_preds)
        predicted_rgb_values = torch.cat(predicted_rgb_values, dim=0)

    # Reshape and convert to numpy array
    predicted_rgb_values = predicted_rgb_values.reshape((height, width, 3))
    predicted_rgb_values = (predicted_rgb_values * 255).numpy().astype(np.uint8)

    # # Load model
    # model = TheModelClass(*args, **kwargs)
    # model.load_state_dict(torch.load(PATH))
    # model.eval()

    # Save model
    torch.save(model.state_dict(), f'./PyTorch Output Images/{optimizer_name}/Q{question_number}_{section_number}/model{model_number}.pth')
    
    # Save the predicted RGB values
    # np.save(f'./PyTorch Output Images/{optimizer_name}/Q{question_number}_{section_number}/predicted_rgb_values_model{model_number}.npy', predicted_rgb_values)

    # Convert to PIL Image and save
    image = Image.fromarray(predicted_rgb_values)
    image.save(f'./PyTorch Output Images/{optimizer_name}/Q{question_number}_{section_number}/image_model{model_number}.png')

In [5]:
# Models

class Model1(nn.Module):
    def __init__(self, input_size):
        super(Model1, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 3)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        return x
    
class Model2(nn.Module):
    def __init__(self, input_size):
        super(Model2, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 32)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(32, 3)
        self.relu4 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.relu4(self.fc4(x))
        return x
    
class Model3(nn.Module):
    def __init__(self, input_size):
        super(Model3, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 64)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(64, 32)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(32, 3)
        self.relu5 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.relu4(self.fc4(x))
        x = self.relu5(self.fc5(x))
        return x

class Model4(nn.Module):
    def __init__(self, input_size):
        super(Model4, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(256, 128)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(128, 64)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(64, 32)
        self.relu5 = nn.ReLU()
        self.fc6 = nn.Linear(32, 3)
        self.relu6 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.relu4(self.fc4(x))
        x = self.relu5(self.fc5(x))
        x = self.relu6(self.fc6(x))
        return x

# Q1

## 1

In [31]:
# Extract coordinates and RGB values from the image
height, width, _ = image_array.shape
coordinates = torch.tensor([(x / width, y / height) for y in range(height) for x in range(width)], dtype=torch.float32)
rgb_values = torch.tensor((image_array.reshape(-1, 3) / 255).astype(np.float32), dtype=torch.float32)

# Split data into training and testing sets
train_coordinates, test_coordinates, train_rgb_values, test_rgb_values = train_test_split(coordinates, rgb_values, test_size=0.2, random_state=42)

# Convert data to DataLoader
train_dataset = TensorDataset(train_coordinates, train_rgb_values)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

input_size = 2

In [ ]:
model1 = Model1(input_size)

model1 = fit_model(model1, train_dataloader)
predict_model(model1, coordinates.detach().clone(), height, width, 1, 1, 1)

In [ ]:
model2 = Model2(input_size)

model2 = fit_model(model2, train_dataloader)
predict_model(model2, coordinates.detach().clone(), height, width, 1, 1, 2)

In [ ]:
model3 = Model3(input_size)

model3 = fit_model(model3, train_dataloader)
predict_model(model3, coordinates.detach().clone(), height, width, 1, 1, 3)

In [ ]:
model4 = Model4(input_size)

model4 = fit_model(model4, train_dataloader)
predict_model(model4, coordinates.detach().clone(), height, width, 1, 1, 4)

## 2

In [10]:
# Extract coordinates and RGB values from the image
height, width, _ = image_array.shape
coordinates = torch.tensor([(x / width, y / height) for y in range(height) for x in range(width)], dtype=torch.float32)
rgb_values = torch.tensor((image_array.reshape(-1, 3) / 255).astype(np.float32), dtype=torch.float32)

# Split data into training and testing sets
train_coordinates, test_coordinates, train_rgb_values, test_rgb_values = train_test_split(coordinates, rgb_values, test_size=0.2, random_state=42)

# Convert data to DataLoader
train_dataset = TensorDataset(train_coordinates, train_rgb_values)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

input_size = 2

In [ ]:
model1 = Model1(input_size)

model1 = fit_model(model1, train_dataloader)
predict_model(model1, coordinates.detach().clone(), height, width, 1, 2, 1)

In [ ]:
model2 = Model2(input_size)

model2 = fit_model(model2, train_dataloader)
predict_model(model2, coordinates.detach().clone(), height, width, 1, 2, 2)

In [ ]:
model3 = Model3(input_size)

model3 = fit_model(model3, train_dataloader)
predict_model(model3, coordinates.detach().clone(), height, width, 1, 2, 3)

In [ ]:
model4 = Model4(input_size)

model4 = fit_model(model4, train_dataloader)
predict_model(model4, coordinates.detach().clone(), height, width, 1, 2, 4)

## 3

In [15]:
# Function to compute gamma using sine and cosine functions
def compute_gamma(coordinates, L):
    x, y = coordinates[:, 0], coordinates[:, 1]
    gama = torch.cat([torch.sin(2**i * np.pi * x).unsqueeze(1) for i in range(L)] +
                     [torch.cos(2**i * np.pi * x).unsqueeze(1) for i in range(L)] +
                     [torch.sin(2**i * np.pi * y).unsqueeze(1) for i in range(L)] +
                     [torch.cos(2**i * np.pi * y).unsqueeze(1) for i in range(L)], dim=1)
    return gama

In [34]:
# Extract coordinates and RGB values from the image
height, width, _ = image_array.shape
coordinates = torch.tensor([(x / width, y / height) for y in range(height) for x in range(width)], dtype=torch.float32)
rgb_values = torch.tensor((image_array.reshape(-1, 3) / 255).astype(np.float32), dtype=torch.float32)

L = 3
coordinates = compute_gamma(coordinates, L)

# Split data into training and testing sets
train_coordinates, test_coordinates, train_rgb_values, test_rgb_values = train_test_split(coordinates, rgb_values, test_size=0.2, random_state=42)

# Convert data to DataLoader
train_dataset = TensorDataset(train_coordinates, train_rgb_values)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

input_size = 4*L

In [ ]:
model1 = Model1(input_size)

model1 = fit_model(model1, train_dataloader)
predict_model(model1, coordinates.detach().clone(), height, width, 1, 3, 1)

In [ ]:
model2 = Model2(input_size)

model2 = fit_model(model2, train_dataloader)
predict_model(model2, coordinates.detach().clone(), height, width, 1, 3, 2)

In [ ]:
model3 = Model3(input_size)

model3 = fit_model(model3, train_dataloader)
predict_model(model3, coordinates.detach().clone(), height, width, 1, 3, 3)

In [ ]:
model4 = Model4(input_size)

model4 = fit_model(model4, train_dataloader)
predict_model(model4, coordinates.detach().clone(), height, width, 1, 3, 4)

# Q2

In [5]:
# Function to compute gamma using sine and cosine functions
def compute_gamma(coordinates, L):
    x, y = coordinates[:, 0], coordinates[:, 1]
    gama = torch.cat([torch.sin(2**i * np.pi * x).unsqueeze(1) for i in range(L)] +
                     [torch.cos(2**i * np.pi * x).unsqueeze(1) for i in range(L)] +
                     [torch.sin(2**i * np.pi * y).unsqueeze(1) for i in range(L)] +
                     [torch.cos(2**i * np.pi * y).unsqueeze(1) for i in range(L)], dim=1)
    return gama

In [13]:
# Extract coordinates and RGB values from the image
height, width, _ = image_array.shape
coordinates = torch.tensor([(x / width, y / height) for y in range(height) for x in range(width)], dtype=torch.float32)
rgb_values = torch.tensor((image_array.reshape(-1, 3) / 255).astype(np.float32), dtype=torch.float32)

L = 3
coordinates = compute_gamma(coordinates, L)

# Split data into training and testing sets
train_coordinates, test_coordinates, train_rgb_values, test_rgb_values = train_test_split(coordinates, rgb_values, test_size=0.2, random_state=42)

# Convert data to DataLoader
train_dataset = TensorDataset(train_coordinates, train_rgb_values)
train_dataloader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

input_size = 4*L

In [ ]:
model1 = Model1(input_size)

model1 = fit_model(model1, train_dataloader)
predict_model(model1, coordinates.detach().clone(), height, width, 2, 3, 1)

In [ ]:
model2 = Model2(input_size)

model2 = fit_model(model2, train_dataloader)
predict_model(model2, coordinates.detach().clone(), height, width, 2, 3, 2)

In [ ]:
model3 = Model3(input_size)

model3 = fit_model(model3, train_dataloader)
predict_model(model3, coordinates.detach().clone(), height, width, 2, 3, 3)

In [ ]:
model4 = Model4(input_size)

model4 = fit_model(model4, train_dataloader)
predict_model(model4, coordinates.detach().clone(), height, width, 2, 3, 4)

# Q3

## 8

In [37]:
# Extract coordinates and RGB values from the image
height, width, _ = image_array.shape
coordinates = torch.tensor([(x / width, y / height) for y in range(height) for x in range(width)], dtype=torch.float32)
rgb_values = torch.tensor((image_array.reshape(-1, 3) / 255).astype(np.float32), dtype=torch.float32)

# Split data into training and testing sets
train_coordinates, test_coordinates, train_rgb_values, test_rgb_values = train_test_split(coordinates, rgb_values, test_size=0.2, random_state=42)

# Convert data to DataLoader
train_dataset = TensorDataset(train_coordinates, train_rgb_values)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

input_size = 2

height, width, _ = tuple(element * 2 for element in image_array.shape)
coordinates_test = torch.tensor([(x / width, y / height) for y in range(height) for x in range(width)], dtype=torch.float32)

In [22]:
# model1 = Model1(input_size)

# model1 = fit_model(optim.SGD, model1, train_dataloader)

# Load model
model1 = Model1(input_size)
model1.load_state_dict(torch.load(f'./PyTorch Output Images/{optimizer_name}/Q1_1/model1.pth'))
model1.eval()

predict_model(model1, coordinates_test.detach().clone(), height, width, 3, 8, 1)

In [23]:
# model2 = Model2(input_size)

# model2 = fit_model(optim.SGD, model2, train_dataloader)

# Load model
model2 = Model2(input_size)
model2.load_state_dict(torch.load(f'./PyTorch Output Images/{optimizer_name}/Q1_1/model2.pth'))
model2.eval()

predict_model(model2, coordinates_test.detach().clone(), height, width, 3, 8, 2)

In [38]:
# model3 = Model3(input_size)

# model3 = fit_model(optim.SGD, model3, train_dataloader)

# Load model
model3 = Model3(input_size)
model3.load_state_dict(torch.load(f'./PyTorch Output Images/{optimizer_name}/Q1_1/model3.pth'))
model3.eval()

predict_model(model3, coordinates_test.detach().clone(), height, width, 3, 8, 3)

In [25]:
# model4 = Model4(input_size)

# model4 = fit_model(optim.SGD, model4, train_dataloader)

# Load model
model4 = Model4(input_size)
model4.load_state_dict(torch.load(f'./PyTorch Output Images/{optimizer_name}/Q1_1/model4.pth'))
model4.eval()

predict_model(model4, coordinates_test.detach().clone(), height, width, 3, 8, 4)

## 9

In [6]:
# Extract coordinates and RGB values from the image
height, width, _ = image_array.shape
coordinates = torch.tensor([(x / width, y / height) for y in range(height) for x in range(width)], dtype=torch.float32)
rgb_values = torch.tensor((image_array.reshape(-1, 3) / 255).astype(np.float32), dtype=torch.float32)

# Split data into training and testing sets
train_coordinates, test_coordinates, train_rgb_values, test_rgb_values = train_test_split(coordinates, rgb_values, test_size=0.2, random_state=42)

# Convert data to DataLoader
train_dataset = TensorDataset(train_coordinates, train_rgb_values)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

input_size = 2

In [7]:
image = Image.fromarray(image_array)

padding_size = (200, 200)
padded_image = ImageOps.expand(image, border=padding_size, fill='white')
padded_image_array = np.array(padded_image)
height, width, _ = padded_image_array.shape
coordinates_padded = torch.tensor([(x / width, y / height) for y in range(height) for x in range(width)], dtype=torch.float32)

In [9]:
# model1 = Model1(input_size)

# model1 = fit_model(optim.SGD, model1, train_dataloader)

# Load model
model1 = Model1(input_size)
model1.load_state_dict(torch.load(f'./PyTorch Output Images/{optimizer_name}/Q1_1/model1.pth'))
model1.eval()

predict_model(model1, coordinates_padded.detach().clone(), height, width, 3, 9, 1)

In [10]:
# model2 = Model2(input_size)

# model2 = fit_model(optim.SGD, model2, train_dataloader)

# Load model
model2 = Model2(input_size)
model2.load_state_dict(torch.load(f'./PyTorch Output Images/{optimizer_name}/Q1_1/model2.pth'))
model2.eval()

predict_model(model2, coordinates_padded.detach().clone(), height, width, 3, 9, 2)

In [11]:
# model3 = Model3(input_size)

# model3 = fit_model(optim.SGD, model3, train_dataloader)

# Load model
model3 = Model3(input_size)
model3.load_state_dict(torch.load(f'./PyTorch Output Images/{optimizer_name}/Q1_1/model3.pth'))
model3.eval()

predict_model(model3, coordinates_padded.detach().clone(), height, width, 3, 9, 3)

In [12]:
# model4 = Model4(input_size)

# model4 = fit_model(optim.SGD, model4, train_dataloader)

# Load model
model4 = Model4(input_size)
model4.load_state_dict(torch.load(f'./PyTorch Output Images/{optimizer_name}/Q1_1/model4.pth'))
model4.eval()

predict_model(model4, coordinates_padded.detach().clone(), height, width, 3, 9, 4)